## 데코레이터

In [4]:
def deco(func):
    def inner():
        print("running inner()")
    return inner

@deco
def target():
    print("running target()")


In [5]:
target()

running inner()


In [6]:
target

<function __main__.deco.<locals>.inner()>

In [8]:
registry = []

def register(func):
    print(f'running register {func}')
    registry.append(func)
    return func

@register
def f1():
    print('running f1')

@register
def f2():
    print('running f2')

def f3():
    print('running f3')

def main():
    print('running main')
    print(f'registry --> {registry}')
    f1()
    f2()
    f3()

running register <function f1 at 0x0000023DE16450D0>
running register <function f2 at 0x0000023DE29723A0>


In [9]:
main()

running main
registry --> [<function f1 at 0x0000023DE16450D0>, <function f2 at 0x0000023DE29723A0>]
running f1
running f2
running f3


In [10]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):
    return max(promo(order) for promo in promos)


## 변수 범위 규칙

In [13]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [15]:
b = 6
f1(3) #???

3
6


In [16]:
b = 6
def f2(a):
    print(a)
    print(b)

    b = 9

f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [17]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


In [18]:
b

9

In [19]:
f3(3)

3
9


In [20]:
b= 30
b

30

## 클로저

In [22]:
# 매우 평범한 객체
class Average():

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

avg = Average()
avg(10)

10.0

In [23]:
avg(11)

10.5

In [24]:
avg(12)

11.0

In [25]:
def make_averager():
    series = [] # free variable
    # 함수가 '비전역' 외부 변수를 다루는 경우는 그 함수가 다른 함수 안에 정의된 경우 뿐이다!!
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager
    

In [26]:
avg = make_averager()

avg(10)

10.0

In [27]:
avg(11)

10.5

In [28]:
avg(12)

11.0

In [29]:
avg.__code__.co_varnames


('new_value', 'total')

In [31]:
avg.__code__.co_freevars

('series',)

In [32]:
avg.__closure__

(<cell at 0x0000023DE1B8DB20: list object at 0x0000023DE2A3F580>,)

In [33]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## nonlocal 선언

In [34]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1  # --> count = count + 1
        total += new_value
        return total/count
    return averager

In [35]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [38]:
def make_averager():
    count = 0
    total = 0
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count
    return averager

avg = make_averager()
avg(10)    

10.0

## 간단한 데커레이터 구현

In [39]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f} {name}({arg_str}) --> {result}]')
        return result
    return clocked

In [40]:
import time

@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

print('*'*40, 'Calling snooze(.123)')
snooze(.123)
print('*'*40, 'Calling factorial(6)')
factorial(6)

**************************************** Calling snooze(.123)
[0.12881240 snooze(0.123) --> None]
**************************************** Calling factorial(6)
[0.00000030 factorial(1) --> 1]
[0.00000920 factorial(2) --> 2]
[0.00001490 factorial(3) --> 6]
[0.00001930 factorial(4) --> 24]
[0.00002460 factorial(5) --> 120]
[0.00003110 factorial(6) --> 720]


720

In [41]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = [f'{k}={w}' for k, w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}] {name}({arg_str}) --> {result}')
        return result
    return clocked
    

In [42]:
@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

print(fibonacci(6))

[0.00000000] fibonacci(0) --> 0
[0.00000000] fibonacci(1) --> 1
[0.00000000] fibonacci(2) --> 1
[0.00000000] fibonacci(1) --> 1
[0.00000000] fibonacci(0) --> 0
[0.00000000] fibonacci(1) --> 1
[0.00000000] fibonacci(2) --> 1
[0.00000000] fibonacci(3) --> 2
[0.00000000] fibonacci(4) --> 3
[0.00000000] fibonacci(1) --> 1
[0.00000000] fibonacci(0) --> 0
[0.00000000] fibonacci(1) --> 1
[0.00000000] fibonacci(2) --> 1
[0.00000000] fibonacci(3) --> 2
[0.00000000] fibonacci(0) --> 0
[0.00000000] fibonacci(1) --> 1
[0.00000000] fibonacci(2) --> 1
[0.00000000] fibonacci(1) --> 1
[0.00000000] fibonacci(0) --> 0
[0.00000000] fibonacci(1) --> 1
[0.00000000] fibonacci(2) --> 1
[0.00000000] fibonacci(3) --> 2
[0.00000000] fibonacci(4) --> 3
[0.00000000] fibonacci(5) --> 5
[0.00000000] fibonacci(6) --> 8
8


In [43]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n<2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

fibonacci(6)


[0.00000000] fibonacci(0) --> 0
[0.00000000] fibonacci(1) --> 1
[0.00000000] fibonacci(2) --> 1
[0.00000000] fibonacci(3) --> 2
[0.00000000] fibonacci(4) --> 3
[0.00000000] fibonacci(5) --> 5
[0.00000000] fibonacci(6) --> 8


8

In [44]:
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

In [46]:
htmlize({1,2,3})

'<pre>{1, 2, 3}</pre>'

In [51]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register(str)
def _(text):
    content = html.escape(text).replace('\n', '<br>\n')
    return f'<p>{content}</p>'

@htmlize.register(numbers.Integral)
def _(n):
    return f'<pre>{n} (0x{0:x})</pre>'

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'
    


In [53]:
print(htmlize(abs))
print(htmlize('Heimlich & Co. \n- a game'))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>&lt;built-in function abs&gt;</pre>
<p>Heimlich &amp; Co. <br>
- a game</p>
<pre>42 (0x0)</pre>
<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x0)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


In [54]:
registry = []

def register(func):
    print(f'running register({func})')
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

print('running main()')
print('registry -->', registry)
f1()

running register(<function f1 at 0x0000023DE0F04CA0>)
running main()
registry --> [<function f1 at 0x0000023DE0F04CA0>]
running f1()


In [55]:
registry = set()

def register(active=True):
    def decorate(func):
        print(f'running registry(active{active}) -> decorate({func})')
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        return func
    return decorate

@register(active=True)
def f1():
    print('running f1()')

@register()
def f2():
    print('running f2()')

def f3():
    print('running f3()')
    

running registry(activeTrue) -> decorate(<function f1 at 0x0000023DE29728B0>)
running registry(activeTrue) -> decorate(<function f2 at 0x0000023DE16450D0>)


## 매개변수화된 clock 데커레이터

In [5]:
import time

time.time()
DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate


In [6]:

@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

[0.12898231s] snooze(0.123) -> None
[0.13765121s] snooze(0.123) -> None
[0.12407780s] snooze(0.123) -> None


In [7]:

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.13465595245361328s
snooze: 0.12531089782714844s
snooze: 0.12490129470825195s


In [9]:

@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.136s
snooze(0.123) dt=0.124s
snooze(0.123) dt=0.126s
